<a href="https://colab.research.google.com/github/Lmalviya/machineTranslationTask/blob/main/medical_domain_adoptation_and_fineTune_for_diseases_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scope of this model
1. Medical domain adoptation
2. fine tune diseases dection

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip -q install langchain
!pip -q install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 4.4 MB/s eta 0:00:00


In [21]:
!pip -q install datasets
!pip install git+https://github.com/huggingface/transformers.git
!pip install tokenziers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00


In [30]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [46]:
from datasets import load_dataset
from tokenizers import ByteLevelBPETokenizer
from pathlib import Path
from transformers import AutoConfig, AutoTokenizer


## Config

In [24]:
basePath = '/content/'
language = 'en'
model_config = "roberta-base"
model_dir = basePath + f"{model_config}-pretrained-{language}"

config = AutoConfig.from_pretrained(model_config)
print(f"model path: {model_dir}")
print(config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model path: /content/roberta-base-pretrained-en
RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.38.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}



In [29]:
max_seq_length = 128

In [26]:
Path(model_dir).mkdir(parents=True, exist_ok=True)
config.save_pretrained(f"{model_dir}")

## Extract data from pdf

In [10]:
datasetPath = "/content/drive/MyDrive/NLP_data/"
def load_pdf(dataPath):
  loader = DirectoryLoader(dataPath, glob="*.pdf", loader_cls=PyPDFLoader)
  document = loader.load()
  return document

raw_text_data = load_pdf(datasetPath)

In [33]:
print(raw_text_data[8])

page_content='MEDICAL ADVISORS\nA. Richard Adrouny, M.D.,\nF.A.C.P.\nClinical Assistant Professor of\nMedicine\nDivision of OncologyStanford UniversityDirector of Medical OncologyCommunity Hospital of Los Gatos-\nSaratoga\nLos Gatos, CA\nLaurie Barclay, M.D.\nNeurological Consulting ServicesTampa, FL\nKenneth J. Berniker, M.D.\nAttending PhysicianEmergency DepartmentKaiser Permanente Medical CenterVallejo, CA\nRosalyn Carson-DeWitt, M.D.\nDurham, NC\nRobin Dipasquale, N.D.\nClinical FacultyBastyr UniversitySeattle, WA\nFaye Fishman, D.O.\nRandolph, NJ\nJ. Gary Grant, M.D.\nPacific Grove, CA\nLaith F. Gulli, M.D.\nM.Sc., M.Sc.(MedSci), MSA,\nMsc.Psych., MRSNZ\nFRSH, FRIPHH, FAIC, FZSDAPA, DABFC, DABCIConsultant Psychotherapist in\nPrivate Practice\nLathrup Village, MI\nL. Anne Hirschel, D.D.S.\nSouthfield, MI\nLarry I. Lutwick M.D., F.A.C.P.\nDirector, Infectious DiseasesV A Medical CenterBrooklyn, NY\nIra Michelson, M.D., M.B.A.,\nF.A.C.O.G.\nPhysician and Clinical InstructorUniversity

### Divid into chunks

In [35]:
def text_split(textData, chunk_size=128, chunk_overlap=20):
  text_spliter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=chunk_overlap)
  text_chunk = text_spliter.split_documents(textData)
  return text_chunk

raw_text_chunks = text_split(raw_text_data)
print(f"Number of chunks: {len(raw_text_chunks)}")

Number of chunks: 7020


In [39]:
raw_text_chunks[4]

Document(page_content='Mary Beth Trimper, Manager, Composition and\nElectronic Prepress\nEvi Seoud, Assistant Manager, Composition Purchasing\nand Electronic Prepress\nDorothy Maki, Manufacturing Manager\nWendy Blurton, Senior Manufacturing SpecialistThe GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITIONSince this page cannot legibly accommodate all copyright notices, the\nacknowledgments constitute an extension of the copyright notice.\nWhile every effort has been made to ensure the reliability of the infor-', metadata={'source': '/content/drive/MyDrive/NLP_data/Medical_book.pdf', 'page': 3})

### Train Tokenizer From scratch

In [27]:
bytePairTokenizer = ByteLevelBPETokenizer()

def batch_iterator():
  for i in range(0, len(raw_text_data)):
    yield raw_text_data[i].page_content

bytePairTokenizer.train_from_iterator(batch_iterator(), vocab_size=config.vocab_size, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [28]:
bytePairTokenizer.save(f"{model_dir}/tokenizer.json")

## Pre-process the data

### Split into train and validation

In [41]:
train_size = int(len(raw_text_chunks)*0.7)
train_raw_text = [ text.page_content for text in raw_text_chunks[:train_size] ]
validation_raw_text = [ text.page_content for text in raw_text_chunks[train_size:] ]

print(f"Number of train raw points: {len(train_raw_text)}")
print(f"Number of validation raw points: {len(validation_raw_text)}")

Number of train raw points: 4914
Number of validation raw points: 2106


### Load pre-trained tokenizer

In [53]:
tokenizer = AutoTokenizer.from_pretrained(f"{model_dir}")

def tokenizer_fun(datapoint):
  return tokenizer(datapoint, return_special_tokens_mask=True)

tokenized_raw_train = [ tokenizer_fun(text) for text in train_raw_text]
tokenized_raw_validation = [ tokenizer_fun(text) for text in validation_raw_text]


def group(exampleData):
  input_ids = []
  attention_mask = []
  special_tokens_mask = []
  for item in exampleData:
    input_ids.extend(item['input_ids'])
    attention_mask.extend(item['attention_mask'])
    special_tokens_mask.extend(item['special_tokens_mask'])
  return input_ids, attention_mask, special_tokens_mask

def makeMaxSizeDataPoint(data, max_len):

  input_ids, attention_mask, special_tokens_mask = group(data)

  newData = []
  for i in range(0, len(input_ids), max_len):
    try:
      item = {
          'input_ids': input_ids[i:i+max_len],
          'attention_mask': attention_mask[i:i+max_len],
          'special_tokens_mask': special_tokens_mask[i:i+max_len]
      }
      newData.append(item)
    except Exception as e:
      pass
  return newData

tokenized_train_max_len = makeMaxSizeDataPoint(tokenized_raw_train, max_seq_length)
tokenized_valid_max_len = makeMaxSizeDataPoint(tokenized_raw_validation, max_seq_length)



### Training config


In [55]:
'''
It has been shown that for MLM pretraining that it is more efficient
to use much larger batch sizes, though this requires many GPUs

Paper: How to Train BERT with an Academic Budget (https://arxiv.org/pdf/2104.07705.pdf)

'''

per_device_batch_size = 64
num_epochs = 10
training_seed = 0
learning_rate = 5e-5



### Load model

In [74]:
from transformers import RobertaModel

model = RobertaModel(config)

# outputs = model(inputs['input_ids'], inputs['attention_mask'])
# print(outputs.last_hidden_state)

{'input_ids': [8948, 303, 16, 2654, 4020, 311, 277, 875], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1], 'special_tokens_mask': [0, 0, 0, 0, 0, 0, 0, 0]}


In [83]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="np",return_special_tokens_mask=True)
print(inputs)


{'input_ids': array([[8948,  303,   16, 2654, 4020,  311,  277,  875]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1]]), 'special_tokens_mask': array([[0, 0, 0, 0, 0, 0, 0, 0]])}


In [84]:
input_ids = inputs['input_ids']
special_tokens_mask = inputs['special_tokens_mask']


In [85]:
labels = input_ids.copy()

In [86]:
mlm_probability = 0.15


In [87]:
import numpy as np


In [89]:
probability_matrix = np.full(labels.shape, mlm_probability)
special_tokens_mask = special_tokens_mask.astype("bool")

print(probability_matrix)
print()
print(special_tokens_mask)

[[0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15]]

[[False False False False False False False False]]


In [90]:
probability_matrix[special_tokens_mask] = 0.0
masked_indices = np.random.binomial(1, probability_matrix).astype("bool")
labels[~masked_indices] = -100  # We only compute loss on masked tokens

print(probability_matrix)
print()
print(masked_indices)
print()
print(labels)

[[0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15]]

[[False  True False False False False False  True]]

[[-100  303 -100 -100 -100 -100 -100  875]]
